In [15]:
#pip install if necessary
!pip install bs4
!pip install numpy
!pip install nltk

In [16]:
# package and libary install
import matplotlib
from IPython.display import Image
matplotlib.use('Agg')
from bs4 import BeautifulSoup
import itertools
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import state_union
from nltk.tag import pos_tag
from nltk.tokenize import PunktSentenceTokenizer
import numpy as np
from operator import itemgetter
import pandas as pd
import requests
import re
from string import punctuation


nltk.download('maxent_treebank_pos_tagger')
nltk.download('treebank')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

pd.options.display.max_colwidth=500

[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package treebank to /home/jovyan/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/jovyan/nltk_data...
[nltk_data]   Package words is already up-to-date!


### 1.	Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8, which together will make up your semester project.   

a.	It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.

b.	Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site. 

c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.  


d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.

In [4]:
def getSoup(url):
    """
    Utility function which takes a url and returns a Soup object.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    return soup

def minMax(a):
    '''Returns the index of negative and positive review.'''
    
    # get the index of least rated user review
    minpos = a.index(min(a))
    
    # get the index of highest rated user review
    maxpos = a.index(max(a))
    
    return minpos, maxpos

def getReviews(soup):
    '''Function returns a negative and positive review for each movie.'''
    
    # get a list of user ratings
    user_review_ratings = [tag.previous_element for tag in 
                           soup.find_all('span', attrs={'class': 'point-scale'})]
    
    
    # find the index of negative and positive review
    n_index, p_index = minMax(list(map(int, user_review_ratings)))
    
    
    # get the review tags
    user_review_list = soup.find_all('a', attrs={'class':'title'})
    
    
    # get the negative and positive review tags
    n_review_tag = user_review_list[n_index]
    p_review_tag = user_review_list[p_index]
    
    # return the negative and positive review link
    n_review_link = "https://www.imdb.com" + n_review_tag['href']
    p_review_link = "https://www.imdb.com" + p_review_tag['href']
    
    return n_review_link, p_review_link

def getReviewText(review_url):
    '''Returns the user review text given the review url.'''
    
    # get the review_url's soup
    soup = getSoup(review_url)
    
    # find div tags with class text show-more__control
    tag = soup.find('div', attrs={'class': 'text show-more__control'})
    
    return tag.getText()

def getMovieTitle(review_url):
    '''Returns the movie title from the review url.'''
    
    # get the review_url's soup
    soup = getSoup(review_url)
    
    # find h1 tag
    tag = soup.find('h1')
    
    return list(tag.children)[1].getText()

def getNounChunks(user_review):
    
    # create the doc object
    doc = nlp(user_review)
    
    # get a list of noun_chunks
    noun_chunks = list(doc.noun_chunks)
    
    # convert noun_chunks from span objects to strings, otherwise it won't pickle
    noun_chunks_strlist = [chunk.text for chunk in noun_chunks]
    
    return noun_chunks_strlist

In [5]:
# limit to 250 movies
#url = '''https://www.imdb.com/search/title/?user_rating=1.0,10.0&genres=comedy&role=nm0001191&sort=user_rating,desc&count=250'''
url = '''https://www.imdb.com/search/title/?title_type=feature&user_rating=1.0,10.0
    &genres=comedy&view=simple&sort=user_rating,desc&count=250&role=nm0001191'''
# get the soup object for main api url
movies_soup = getSoup(url)

In [6]:
# find all a-tags with class:None
movie_tags = movies_soup.find_all('a', attrs={'class': None})

# filter the a-tags to get just the titles
movie_tags = [tag.attrs['href'] for tag in movie_tags 
              if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]

# remove duplicate links
movie_tags = list(dict.fromkeys(movie_tags))

In [7]:
# movie links
base_url = "https://www.imdb.com"
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]

In [8]:
# get a list of soup objects
movie_soups = [getSoup(link) for link in movie_links]

# get all 500 movie review links
movie_review_list = [getReviews(movie_soup) for movie_soup in movie_soups]
movie_review_list = list(itertools.chain(*movie_review_list))

#### 2.	Extract noun phrase (NP) chunks from your reviews using the following procedure:

a.	In Python, use BeautifulSoup to grab the main review text from each link. 

b.	Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser. 

c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.

In [9]:
# get review text from the review link
review_texts = [getReviewText(url) for url in movie_review_list]

# get movie name from the review link
movie_titles = [getMovieTitle(url) for url in movie_review_list]

# construct a dataframe
df = pd.DataFrame({'movie': movie_titles, 'user_review_permalink': movie_review_list, 'user_review': review_texts})

In [10]:
df['user_review'].dropna(inplace=True)
df['tokened'] = df['user_review'].apply(word_tokenize)

In [11]:
word_list=[]
for i in range(len(df['tokened'])):
    word_list.append([])
for i in range(len(df['tokened'])): 
    for word in df['tokened'][i]:
        if word[0:].isalpha():
            word_list[i].append(word[0:]) 

In [12]:
tagged_tokens=[]
for token in word_list:
    tagged_tokens.append(nltk.pos_tag(token))

In [13]:
tagged_tokens

[[('Adam', 'NNP'),
  ('Sandler', 'NNP'),
  ('is', 'VBZ'),
  ('known', 'VBN'),
  ('for', 'IN'),
  ('his', 'PRP$'),
  ('wacky', 'JJ'),
  ('flaky', 'NN'),
  ('and', 'CC'),
  ('sometimes', 'RB'),
  ('over', 'IN'),
  ('the', 'DT'),
  ('top', 'JJ'),
  ('style', 'NN'),
  ('of', 'IN'),
  ('comedy', 'NN'),
  ('With', 'IN'),
  ('films', 'NNS'),
  ('like', 'IN'),
  ('Billy', 'NNP'),
  ('Madison', 'NNP'),
  ('Happy', 'NNP'),
  ('Gilmore', 'NNP'),
  ('and', 'CC'),
  ('Big', 'NNP'),
  ('Daddy', 'NNP'),
  ('under', 'IN'),
  ('his', 'PRP$'),
  ('belt', 'NN'),
  ('one', 'CD'),
  ('would', 'MD'),
  ('think', 'VB'),
  ('that', 'IN'),
  ('he', 'PRP'),
  ('would', 'MD'),
  ('stick', 'VB'),
  ('with', 'IN'),
  ('what', 'WP'),
  ('he', 'PRP'),
  ('got', 'VBD'),
  ('Sandler', 'NNP'),
  ('has', 'VBZ'),
  ('shown', 'VBN'),
  ('us', 'PRP'),
  ('a', 'DT'),
  ('new', 'JJ'),
  ('side', 'NN'),
  ('of', 'IN'),
  ('him', 'PRP'),
  ('with', 'IN'),
  ('his', 'PRP$'),
  ('performance', 'NN'),
  ('in', 'IN'),
  ('Paul', '

In [18]:
pattern = "NP: {<DT>?<JJ>*<NN>}"
NPChunker = nltk.RegexpParser(pattern) 
chunked = NPChunker.parse(tagged_tokens[0])
print(chunked)

(S
  Adam/NNP
  Sandler/NNP
  is/VBZ
  known/VBN
  for/IN
  his/PRP$
  (NP wacky/JJ flaky/NN)
  and/CC
  sometimes/RB
  over/IN
  (NP the/DT top/JJ style/NN)
  of/IN
  (NP comedy/NN)
  With/IN
  films/NNS
  like/IN
  Billy/NNP
  Madison/NNP
  Happy/NNP
  Gilmore/NNP
  and/CC
  Big/NNP
  Daddy/NNP
  under/IN
  his/PRP$
  (NP belt/NN)
  one/CD
  would/MD
  think/VB
  that/IN
  he/PRP
  would/MD
  stick/VB
  with/IN
  what/WP
  he/PRP
  got/VBD
  Sandler/NNP
  has/VBZ
  shown/VBN
  us/PRP
  (NP a/DT new/JJ side/NN)
  of/IN
  him/PRP
  with/IN
  his/PRP$
  (NP performance/NN)
  in/IN
  Paul/NNP
  Thomas/NNP
  Anderson/NNP
  Punch/NNP
  Drunk/NNP
  plays/VBZ
  Barry/NNP
  Egan/NNP
  (NP an/DT ambitious/JJ man/NN)
  who/WP
  owns/VBZ
  and/CC
  operates/VBZ
  his/PRP$
  (NP own/JJ business/NN)
  He/PRP
  sells/VBZ
  his/PRP$
  own/JJ
  plungers/NNS
  One/CD
  (NP day/NN)
  after/IN
  finishing/VBG
  (NP a/DT phone/NN)
  (NP call/NN)
  with/IN
  (NP a/DT food/NN)
  (NP company/NN)
  he/PRP
  

In [19]:
new_words = ['Adam', 'Sandler', 'Happy', 'Gilmore', 'Big', 'Daddy', 'Billy', 'Madison', 'Punch', 'Drunk', 'Drew', 'Barrymore', 'Rob', 'Schneider', 'Peter', 'Segal']

In [24]:
stopwords = nltk.corpus.words.words()
stopwords.append(new_words)

## 3.	Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).


#### Tokens

In [25]:
word_list=[]
for i in range(len(df['tokened'])):
    word_list.append([])
for i in range(len(df['tokened'])): 
    for word in df['tokened'][i]:
        if word[0:].isalpha():
            word_list[i].append(word[0:]) 

In [26]:
tagged_tokens=[]
for token in word_list:
    tagged_tokens.append(nltk.pos_tag(token))

In [27]:
tagged_tokens

[[('Adam', 'NNP'),
  ('Sandler', 'NNP'),
  ('is', 'VBZ'),
  ('known', 'VBN'),
  ('for', 'IN'),
  ('his', 'PRP$'),
  ('wacky', 'JJ'),
  ('flaky', 'NN'),
  ('and', 'CC'),
  ('sometimes', 'RB'),
  ('over', 'IN'),
  ('the', 'DT'),
  ('top', 'JJ'),
  ('style', 'NN'),
  ('of', 'IN'),
  ('comedy', 'NN'),
  ('With', 'IN'),
  ('films', 'NNS'),
  ('like', 'IN'),
  ('Billy', 'NNP'),
  ('Madison', 'NNP'),
  ('Happy', 'NNP'),
  ('Gilmore', 'NNP'),
  ('and', 'CC'),
  ('Big', 'NNP'),
  ('Daddy', 'NNP'),
  ('under', 'IN'),
  ('his', 'PRP$'),
  ('belt', 'NN'),
  ('one', 'CD'),
  ('would', 'MD'),
  ('think', 'VB'),
  ('that', 'IN'),
  ('he', 'PRP'),
  ('would', 'MD'),
  ('stick', 'VB'),
  ('with', 'IN'),
  ('what', 'WP'),
  ('he', 'PRP'),
  ('got', 'VBD'),
  ('Sandler', 'NNP'),
  ('has', 'VBZ'),
  ('shown', 'VBN'),
  ('us', 'PRP'),
  ('a', 'DT'),
  ('new', 'JJ'),
  ('side', 'NN'),
  ('of', 'IN'),
  ('him', 'PRP'),
  ('with', 'IN'),
  ('his', 'PRP$'),
  ('performance', 'NN'),
  ('in', 'IN'),
  ('Paul', '

#### Chunker

In [28]:
pattern = "NP: {<DT>?<JJ>*<NN>}"
NPChunker = nltk.RegexpParser(pattern) 
chunked = NPChunker.parse(tagged_tokens[0])
print(chunked)

(S
  Adam/NNP
  Sandler/NNP
  is/VBZ
  known/VBN
  for/IN
  his/PRP$
  (NP wacky/JJ flaky/NN)
  and/CC
  sometimes/RB
  over/IN
  (NP the/DT top/JJ style/NN)
  of/IN
  (NP comedy/NN)
  With/IN
  films/NNS
  like/IN
  Billy/NNP
  Madison/NNP
  Happy/NNP
  Gilmore/NNP
  and/CC
  Big/NNP
  Daddy/NNP
  under/IN
  his/PRP$
  (NP belt/NN)
  one/CD
  would/MD
  think/VB
  that/IN
  he/PRP
  would/MD
  stick/VB
  with/IN
  what/WP
  he/PRP
  got/VBD
  Sandler/NNP
  has/VBZ
  shown/VBN
  us/PRP
  (NP a/DT new/JJ side/NN)
  of/IN
  him/PRP
  with/IN
  his/PRP$
  (NP performance/NN)
  in/IN
  Paul/NNP
  Thomas/NNP
  Anderson/NNP
  Punch/NNP
  Drunk/NNP
  plays/VBZ
  Barry/NNP
  Egan/NNP
  (NP an/DT ambitious/JJ man/NN)
  who/WP
  owns/VBZ
  and/CC
  operates/VBZ
  his/PRP$
  (NP own/JJ business/NN)
  He/PRP
  sells/VBZ
  his/PRP$
  own/JJ
  plungers/NNS
  One/CD
  (NP day/NN)
  after/IN
  finishing/VBG
  (NP a/DT phone/NN)
  (NP call/NN)
  with/IN
  (NP a/DT food/NN)
  (NP company/NN)
  he/PRP
  

#### I collected 250 reviews from IMDB that were in the comedy genre and one of the actors was Adam Sandler. I used a web crawler to grab the links for each review and then parse out the html from inside of it. Here's the link I used to help facilitate the crawling: https://shravan-kuchkula.github.io/scrape_imdb_movie_reviews/#construct-a-dataframe